In [1]:
from pap2 import *

# Versuch 241: RCL-Glied
## 1. Bestimmung der Zeitkonstante

In [4]:
C, R, T12, dT12 = loadtxt("241\\1.csv", skiprows=1, delimiter=',',unpack=True)
C = un.uarray(C, 0.1*C)
R = un.uarray(R, 0.05*R)
f = array([120,120,320])
T12 = un.uarray(T12, dT12)

t_theo = R*C
t_exp = T12/log(2)

a1 = Table("Tabelle 1: Bestimmung der Zeitkonstante eines RC-Glieds")
a1.field_names = ["$C \ [nF]$", "$R \ [k \Omega]$", "$f \ [Hz]$", "$\\tau_{exp} \ [s]$", "$\\tau_{theo} \ [s]$"]
for i in range(3):
    a1.add_row(pformat([C[i], R[i], f[i], t_exp[i], t_theo[i]]))
    
display_table(a1)

$C \ [nF]$,$R \ [k \Omega]$,$f \ [Hz]$,$\tau_{exp} \ [s]$,$\tau_{theo} \ [s]$
$\left(4.7 \pm 0.5\right) \times 10^{2}$,$1.00 \pm 0.05$,120,$0.462 \pm 0.029$,$\left(4.7 \pm 0.5\right) \times 10^{2}$
$4.7 \pm 0.5$,$10.0 \pm 0.5$,120,$0.058 \pm 0.007$,$47 \pm 5$
$47 \pm 5$,$1.00 \pm 0.05$,320,$0.0491 \pm 0.0029$,$47 \pm 5$
